In [ ]:
!pip install sentence-transformers lancedb

In [3]:
import lancedb
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer("intfloat/multilingual-e5-large")

In [5]:
db = lancedb.connect("./lancedb")

In [13]:
import pyarrow as pa

arrow_schema = pa.schema([
    pa.field("text", pa.utf8()),
    pa.field("vector", pa.list_(pa.float32(), 1024)),
])

In [14]:
# from lancedb.pydantic import LanceModel, Vector

# class LanceModelSchema(LanceModel):
#     text: str
#     vector: Vector(1024)

In [15]:
table = db.create_table("test", schema=arrow_schema, mode="overwrite")

In [16]:
table.add(
    [
      {
        "text": "Рим е столица на Италия.",
        "vector": encoder.encode("Рим е столица на Италия.")
      }
    ]
)

table.add(
    [
      {
        "text": "Rome is the capital city of Italy.",
        "vector": encoder.encode("Rome is the capital city of Italy.")
      }
    ]
)

table.add(
    [
      {
        "text": "Овчарските кучета са добри пазачи.",
        "vector": encoder.encode("Овчарските кучета са добри пазачи.")
      }
    ]
)

In [17]:
import pandas as pd

result = table.search(encoder.encode("столица на Италия").tolist()).limit(5).to_pandas()

display(result)

,text,vector,_distance
0,Рим е столица на Италия.,"[0.022451911, 0.0107246665, 0.0008066586, -0.0...",0.182481
1,Rome is the capital city of Italy.,"[0.024205344, 0.024059003, -0.013217255, 0.000...",0.330494
2,Овчарските кучета са добри пазачи.,"[0.02244455, 0.0014089615, -0.032290105, -0.03...",0.530019
